In [2]:
import os
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from torch import Tensor
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
from tqdm import tqdm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from PIL import Image
import cv2
import random
# import torchaudio

In [3]:
mylist_1= os.listdir('data/')
type(mylist_1)
mylist=[]
for item in mylist_1:
    if item[-3:]=='wav':
        mylist.append(item)

df = pd.read_csv('./aihub/AIhub.csv')

,path,emotion
0,./4y1/5e367ec8ee8206179943c051.wav,angry
1,./4y1/5e37d20fc8c25f16cd145708.wav,angry
2,./4y1/5e36610e7995ef170fc0e868.wav,angry
3,./4y1/5e41317c0b3f160367c2075c.wav,angry
4,./4y1/5e426f780b3f160367c2097d.wav,angry
...,...,...
2190,./5y2/5f7124ddf8fac448cc0a6229.wav,sadness
2191,./4y1/5e33ce7b5807b852d9e0580f.wav,sadness
2192,./4y1/5e43a429189842034d9f77a7.wav,sadness
2193,./4y1/5e44f7f0f2fe7a3b1d544784.wav,sadness


In [23]:
#wav2spec
class ETRIDataset_spectram():
    def __init__(self, file_list,dataframe ,frame_length=0.025, frame_stride=0.010):
        self.file_list = file_list
        self.dataframe = dataframe
        self.frame_length = frame_length
        self.frame_stride = frame_stride
    
    def __len__(self):
        return (len(self.file_list) + len(self.dataframe))

    def __getitem__(self,index):
        if index < len(self.file_list):
            audio_path = self.file_list[index]
            X, sample_rate = librosa.load('data/'+audio_path, res_type='kaiser_fast',sr=16000,offset=0.0)
        else:
            index_2 = index-len(self.file_list)
            audio_path = self.dataframe['path'][index_2]
            X, sample_rate = librosa.load('aihub/'+audio_path[6:], res_type='kaiser_fast',sr=16000,offset=0.5)
        sample_rate = sample_rate
        input_nfft = int(round(sample_rate*self.frame_length))
        input_stride = int(round(sample_rate*self.frame_stride))

        S = librosa.feature.melspectrogram(y=X, n_mels=64, n_fft=input_nfft, hop_length=input_stride)
        P = librosa.power_to_db(S, ref=np.max)
        

        ## get label
        if index < len(self.file_list):
            if audio_path[-3:] == 'wav':
                if audio_path[7:8] =='a':
                    label = 0
                elif audio_path[7:8] =='n':
                    label = 1
                elif audio_path[7:8] =='s':
                    label = 2
                elif audio_path[7:8] =='h':
                    label = 3
            else:
                label=None
        else:
            index_2 = index-len(self.file_list)
            if audio_path[-3:] == 'wav':
                if df['emotion'][index_2] == 'angry':
                    label = 0
                elif df['emotion'][index_2] == 'neutral':
                    label = 1
                elif df['emotion'][index_2] == 'sadness':
                    label = 2
                elif df['emotion'][index_2] == 'happiness':
                    label = 3
            else:
                label=None
        return P, label

In [25]:
# train,test,val split
train_size = int(0.8*(len(df) + len(mylist)))

val_size = int((len(df) + len(mylist)-train_size))
train_set, val_set = torch.utils.data.random_split(ETRIDataset_spectram(mylist,df),[train_size,val_size])
print(train_size)
print(val_size)


4060
1015


In [26]:
# spec2img
def getimg(dataset,feature):
        img_path=[]
        labels=[]
        for i in range(len(dataset)):
                fig = plt.figure()
                ax = fig.add_subplot(111)
                p = librosa.display.specshow(dataset[i][0],ax=ax, sr=16000, hop_length=int(round(16000*0.025)), x_axis='time',y_axis='linear')
                extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())

                fig.savefig('./image/%s_%d_%s.jpg' % (dataset[i][1],i,feature), bbox_inches=extent)
                img_path.append('./image/%s_%d_%s.jpg' % (dataset[i][1],i,feature))
                labels.append(dataset[i][1])
                plt.ioff()
                plt.close()
        return img_path , labels


In [27]:
train_path , train_labels = getimg(train_set,'train')
val_path , val_labels = getimg(val_set,'val')

In [28]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
                                                           

In [29]:
class img2tensor():
    def __init__(self,data_path,labels,transforms):
        self.data_path = data_path
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.data_path)

        
    def __getitem__(self, index):
        img_path = self.data_path[index]
        image = Image.open(img_path)
        I = train_transforms(image)
        label = self.labels[index]

        return I, label
        
        

In [30]:
# set batch_size
batch_size = 16
# dataloader
train_dataloader = torch.utils.data.DataLoader(img2tensor(train_path,train_labels,train_transforms), batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(img2tensor(val_path,val_labels,test_transforms), batch_size=batch_size, shuffle=False)
dataloaders_dict ={'train':train_dataloader, 'val': val_dataloader}
# test
batch_iterator = iter(dataloaders_dict['train'])
inputs, labels = next(batch_iterator)
print(inputs.size())
print(labels)

torch.Size([16, 3, 224, 224])
tensor([2, 2, 2, 0, 0, 2, 2, 2, 3, 0, 0, 0, 3, 0, 1, 0])


In [47]:
model = torchvision.models.densenet121(pretrained=False)
model.to(torch.device('cuda'))
model.classifier = nn.Linear(in_features=1024, out_features=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters() ,lr=0.00001, weight_decay=1e-6, momentum=0.9)
model.train()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [48]:
def train(net, dataloader, criterion, optimizer, num_epochs):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)
    model.to(device)
    torch.backends.cudnn.benchmark = True

    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1,num_epochs))
        print('-------------------------------')

        for phase in ['train','val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch == 0) and(phase == 'train'):
                continue
            for inputs, labels in tqdm(dataloader[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs,1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() *inputs.size(0)

                    epoch_corrects += torch.sum(preds == labels.data)

                    epoch_loss = epoch_loss / len(dataloader[phase].dataset)
                    epoch_acc = epoch_corrects.double() / len(dataloader[phase].dataset)


            print('{} Loss: {:.4f} ACC {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
                

In [49]:
train(model, dataloaders_dict, criterion,optimizer, num_epochs=200)

cuda:0
Epoch 1/200
-------------------------------


100%|██████████| 64/64 [00:03<00:00, 16.61it/s]


val Loss: 0.0107 ACC 0.2296
Epoch 2/200
-------------------------------


100%|██████████| 254/254 [00:32<00:00,  7.76it/s]


train Loss: 0.0038 ACC 0.3692


...

val Loss: 0.0158 ACC 0.6089
Epoch 200/200
-------------------------------


100%|██████████| 254/254 [00:31<00:00,  8.02it/s]


train Loss: 0.0017 ACC 0.8022


100%|██████████| 64/64 [00:03<00:00, 16.97it/s]

val Loss: 0.0113 ACC 0.5970


In [50]:
# save weight
save_path = './saved_models/DenseNet121_img_aihub+etri.pth'
torch.save(model.state_dict(),save_path)

In [51]:
# load weight
load_path = './saved_models/DenseNet121_img_aihub+etri.pth'
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
model = torchvision.models.densenet121(pretrained=False)
# first_conv_layer = [nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)]
# first_conv_layer.extend(list(model.features))  
# model.features= nn.Sequential(*first_conv_layer )  
model.classifier = nn.Linear(in_features=1024, out_features=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters() ,lr=0.00001, weight_decay=1e-6, momentum=0.9)
model.eval()
model.load_state_dict(load_weights)

<All keys matched successfully>

In [52]:
class Predictor(object):
    def __init__(self, model, device ='cpu',  fp16=False ):
        self.model = model
        
        self.cls_name = {0:'angry', 1:'happy', 2:'sad', 3:'neutral'}
        self.device = device
        

    def predict(self, audio):
        
        audio_info = audio
        
        outputs = self.model(audio_info)
        probability = torch.softmax(outputs,1)
        probability = probability.squeeze()
        proba, idx = torch.max(probability, dim=0)
        emo_proba = proba.item()
        print(emo_proba)
        idx = idx.item()
        emo_label = self.cls_name[idx]
        print(emo_label)
        return emo_label



In [53]:
def swdata(emotion):
    swlist_1 = os.listdir('swdata/%s/' % emotion)
    type(swlist_1)
    swlist = []
    for item in swlist_1:
        if item[-3:] == 'wav':
            swlist.append('./swdata/%s/' % emotion + item)
    return swlist

In [54]:
sw_list = swdata('angry') + swdata('happy') + swdata('sad') + swdata('neutral')

random.shuffle(sw_list)

In [55]:
#wav2spec
class swdata_spectram():
    def __init__(self, file_list, frame_length=0.025, frame_stride=0.010):
        self.file_list = file_list
        self.frame_length = frame_length
        self.frame_stride = frame_stride
    
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self,index):
        audio_path = self.file_list[index]
        X, sample_rate = librosa.load(audio_path, res_type='kaiser_fast',duration=2.5,sr=16000,offset=0.0)
        signal = np.zeros((int(sample_rate *3,)))
        signal[:len(X)] = X
        sample_rate = sample_rate
        input_nfft = int(round(sample_rate*self.frame_length))
        input_stride = int(round(sample_rate*self.frame_stride))

        S = librosa.feature.melspectrogram(y=X, n_mels=64, n_fft=input_nfft, hop_length=input_stride)
        P = librosa.power_to_db(S, ref=np.max)
        

        ## get label
        if audio_path[-3:] == 'wav':
            if audio_path[9:10] =='a':
                label = 0
            elif audio_path[9:10] =='n':
                label = 1
            elif audio_path[9:10] =='s':
                label = 2
            elif audio_path[9:10] =='h':
                label = 3
        else:
            label=None
        return P, label

In [56]:
test_swdata = swdata_spectram(sw_list)
test_path , test_labels = getimg(test_swdata,'test_sw')


In [57]:
test_path

['./image/0_0_test_sw.jpg',
 './image/1_1_test_sw.jpg',
 './image/3_2_test_sw.jpg',
 './image/0_3_test_sw.jpg',
 './image/1_4_test_sw.jpg',
 './image/0_5_test_sw.jpg',
 './image/2_6_test_sw.jpg',
 './image/3_7_test_sw.jpg',
 './image/3_8_test_sw.jpg',
 './image/1_9_test_sw.jpg',
 './image/3_10_test_sw.jpg',
 './image/1_11_test_sw.jpg',
 './image/3_12_test_sw.jpg',
 './image/3_13_test_sw.jpg',
 './image/3_14_test_sw.jpg',
 './image/2_15_test_sw.jpg',
 './image/2_16_test_sw.jpg',
 './image/0_17_test_sw.jpg',
 './image/0_18_test_sw.jpg',
 './image/0_19_test_sw.jpg',
 './image/2_20_test_sw.jpg',
 './image/3_21_test_sw.jpg',
 './image/3_22_test_sw.jpg',
 './image/1_23_test_sw.jpg',
 './image/2_24_test_sw.jpg',
 './image/2_25_test_sw.jpg',
 './image/2_26_test_sw.jpg',
 './image/1_27_test_sw.jpg',
 './image/2_28_test_sw.jpg',
 './image/1_29_test_sw.jpg',
 './image/3_30_test_sw.jpg',
 './image/2_31_test_sw.jpg',
 './image/2_32_test_sw.jpg',
 './image/0_33_test_sw.jpg',
 './image/2_34_test_sw.j

In [58]:
img2tensor(test_path,test_labels,test_transforms)[1][0].unsqueeze(0).shape

torch.Size([1, 3, 224, 224])

In [59]:
predictor = Predictor(model)
device='cpu'
a=[]
b=[]
for i in range(len(img2tensor(test_path,test_labels,test_transforms))):
    a.append(predictor.predict(img2tensor(test_path,test_labels,test_transforms)[i][0].unsqueeze(0)))
    b.append(img2tensor(test_path,test_labels,test_transforms)[i][1])


0.4958884119987488
sad
0.9950120449066162
happy
0.7302796840667725
angry
0.45183807611465454
happy
0.9572561383247375
neutral
0.6266244649887085
happy
0.4768260419368744
happy
0.9355182647705078
neutral
0.9780303239822388
neutral
0.8595733642578125
neutral
0.8648070693016052
neutral
0.983895480632782
neutral
0.342019647359848
neutral
0.9802371859550476
happy
0.9696584939956665
sad
0.49740564823150635
neutral
0.6623514294624329
sad
0.5168517827987671
angry
0.6791277527809143
neutral
0.41617104411125183
neutral
0.560595691204071
happy
0.45584866404533386
happy
0.6148668527603149
happy
0.8141850233078003
neutral
0.5241822004318237
sad
0.6362661719322205
neutral
0.4846092760562897
happy
0.3839997351169586
happy
0.7492615580558777
sad
0.7233368158340454
angry
0.7482103109359741
happy
0.5566442608833313
happy
0.6820038557052612
happy
0.9881565570831299
neutral
0.7938016057014465
neutral
0.6277292966842651
happy
0.7981257438659668
sad
0.9441376328468323
angry
0.5681082606315613
neutral
0.4926

In [60]:
df = pd.DataFrame(a,columns=['predict'])
df['original']=b
for i in range(len(df)):
    if df['original'][i] == 0:
        df['original'][i] = 'angry'
    elif df['original'][i] == 1:
        df['original'][i] = 'happy'
    elif df['original'][i] == 2:
        df['original'][i] = 'sad'
    elif df['original'][i] == 3:
        df['original'][i] = 'neutral'
print('accuracy={}'.format((df['predict']==df['original']).sum()/len(df)))

accuracy=0.2808988764044944


/tmp/ipykernel_21010/4207275683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['original'][i] = 'angry'
/home/seojungin/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [61]:
df.to_csv('predict_result_img_sw_ai+etri.csv',index=False)

In [46]:
import seaborn as sns
